In [0]:
import pandas as pd

In [0]:
# Open the data file of Incendie and House With Centroid
path = 'Data/'
file1 = path + 'houseWithCentroid.csv'
file2 = path + 'donneesouvertes-interventions-sim.csv'
data1 = pd.read_csv(file1)
data2 = pd.read_csv(file2)


In [0]:
# Select the useful data from House With Centroid, and drop the data with null values.
house_centroid_withna = data1[['Unnamed: 0', 'ID_UEV', 'ANNEE_CONSTRUCTION', 'centroid']]
house_centroid = house_centroid_withna.dropna(axis = 'index', how = 'any')

In [0]:
# The data of longitudes and latitudes in original Dataset are saved as object. These functions 
#     can read longitude and latitude from the form House With Centroid.
# function point_to_coordinate: load longitude and latitude from Point Structure.
import re
def point_to_coordinate(point_string):
    strs = re.findall(r"-?\d+\.?\d*", point_string)
    numbers = [float(s) for s in strs]
    return numbers

# function geometry_to_coordinate: load multi longitudes and latitudes from Geometry Structure
def geometry_to_coordinates(geometry_string):
    point_strings = re.findall(r"-?\d+\.?\d*\s-?\d+\.?\d*", geometry_string)
    double_number_list = [point_to_coordinate(dn) for dn in point_strings]
    return double_number_list


In [0]:

import math
# function radians: Turn the 360 degree to 2 Pi radians value.
def radians(degree):
    return degree * 3.141592653589793 / 180.0;

# function distance: Compute the distance of two points with their longitudes and latitudes
#      this function only works in short distance.
def distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    dy = radians(y2 - y1) * 6371
    dx = radians(x2 - x1) * 6371 * (math.cos( radians(y1) ) + math.cos( radians(y2) )) /2
    return math.sqrt(dy*dy + dx * dx)


In [0]:
# If we compare every incindie and every house, there will be serveral T amounts of records! So we 
#      split the area we concerned into 400 * 400 grids and stored the house data into them.
min_lat = 45.40
max_lat = 45.80
min_long = -74.00
max_long = -73.40

scale_lat = 400
scale_long = 400

# function getY: Decide the Y coordinate of Grid with the latitude info.min_lat = 45.40
def getY(latitude):
    return int((latitude - min_lat) / (max_lat - min_lat) * scale_lat)

# function getX: Decide the X coordinate of Grid with the longitude info.
def getX(longitude):
    return int((longitude - min_long) / (max_long - min_long ) * scale_long)


In [0]:
# Grids for storing the house info
house_map = []
for i in range(scale_long):
    house_map.append([])
    for j in range(scale_lat):
        house_map[i].append([])

# Store the house info into the grids
n = 0
for record in house_centroid.values:
    longitude, latitude = point_to_coordinate(record[3])
    i = getX(longitude)
    j = getY(latitude)
    n = n+1
    house_map[i][j].append((record[0], record[1], int(record[2]), longitude, latitude))

In [0]:
# Select the useful data from Incident.
alert_info = data2[['incident_nbr', 'creation_date_time', 'incident_type_desc',  'description_groupe', 'longitude', 'latitude']]

In [0]:
# For every incident, find the (house_num: 5) nearest house and store their distance and house age.
alert_house_va = []

house_num = 5

for alert in alert_info.values:
    alert_longitude = float(alert[4])
    alert_latitude = float(alert[5])
    
    # Calculate which grid the incident is in.
    i = getX(alert_longitude)
    j = getY(alert_latitude)

    # We had stored the house info into the grids. So we need only compare the house in the 
    #      grid where the incident in and the 8 nearby grids.
    temp = []
    for x in range(max(0, i-1), min(scale_long, i+2)):
        for y in range(max(0, j-1), min(scale_lat, j+2)):
            for house in house_map[x][y]:
                house_longitude = house[3]
                house_latitude = house[4]
                age = int(alert[1][0:4]) - house[2]
                d = distance((alert_longitude, alert_latitude), (house_longitude, house_latitude)) *1000
                # If the age is negative, that means the house did not exist when the incident happened
                # We don't consider about the distance larger than 100 meters by now.
                if d < 50 and age >=0:
                    record = (alert[0], alert[1], alert[2], alert[3], alert[4], alert[5], house[0], house[1], house[2], house[3], house[4], age, d)
                    for i in range(min(house_num, len(temp))):
                        if record[-1] < temp[i][-1]:
                            temp.insert(i, record)
                            break
                    else:
                        temp.append(record)
                        
    # Select the first (house_num: 5) house in the list. 
    for i in range(min(house_num, len(temp))):
        alert_house_va.append(temp[i])
#
len(alert_house_va)

1479239

In [0]:
alert_house = pd.DataFrame(alert_house_va)
alert_house.columns = ['incident_nbr', 'incident_time', 'incident_type_desc',  'description_groupe', 'incident_long', 'incident_lat', 'house_idx', 'house_id_uev', 'house_build_year', 'house_long', 'house_lat',  'house_age', 'distance']


In [0]:
alert_house.describe()

,incident_nbr,incident_long,incident_lat,house_idx,house_id_uev,house_build_year,house_long,house_lat,house_age,distance
count,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06,1.479239e+06
mean,6.137502e+04,-7.361688e+01,4.552851e+01,2.393747e+05,2.824104e+06,1.955053e+03,-7.361688e+01,4.552850e+01,6.109757e+01,3.238917e+01
std,3.857990e+04,7.839566e-02,5.588166e-02,1.515082e+05,1.386539e+06,3.260101e+01,7.839026e-02,5.587846e-02,3.260929e+01,7.472253e+00
min,1.000000e+00,-7.396768e+01,4.540269e+01,4.000000e+00,1.000002e+06,1.680000e+03,-7.396755e+01,4.540234e+01,0.000000e+00,3.925600e+00
25%,2.600800e+04,-7.363865e+01,4.548979e+01,1.039350e+05,2.016234e+06,1.934000e+03,-7.363861e+01,4.548974e+01,4.000000e+01,2.686708e+01
50%,5.988100e+04,-7.359990e+01,4.552644e+01,2.326940e+05,3.003164e+06,1.958000e+03,-7.359996e+01,4.552650e+01,5.800000e+01,3.105387e+01
75%,9.501200e+04,-7.356890e+01,4.556395e+01,3.700000e+05,4.038055e+06,1.976000e+03,-7.356895e+01,4.556395e+01,8.300000e+01,3.740992e+01
max,1.316730e+05,-7.347940e+01,4.570178e+01,5.132320e+05,5.256633e+06,2.017000e+03,-7.347949e+01,4.570213e+01,3.380000e+02,4.999975e+01


In [0]:
# prepare the incident information we need and we will combine them with the house info later.
alert_house_30_short = alert_house[['incident_nbr', 'incident_time', 'incident_type_desc',  'description_groupe', 'incident_long', 'incident_lat']]
alert_house_30_simple = alert_house_30_short.drop_duplicates(['incident_nbr', 'incident_time'])

alert_house_30_mean = alert_house_30_simple.set_index(['incident_nbr', 'incident_time'])

In [0]:
# For each incident, compute the mean house age of the 5 nearest house
mean_value = alert_house.groupby(['incident_time','incident_nbr' ])['house_age'].mean().reset_index()
alert_mean_value = mean_value.set_index(['incident_nbr', 'incident_time'])

In [0]:
# Combine the incident info and average house age.
alert_house_30_mean['mean_age'] = alert_mean_value['house_age']

alert_house_30_mean

,,incident_type_desc,description_groupe,incident_long,incident_lat,mean_age
incident_nbr,incident_time,,,,,
1,2015-01-01 00:03:22,Inondation,Sans incendie,-73.580575,45.535698,99.800000
2,2015-01-01 00:05:58,Ac.véh./1R/s.v./ext/29B/D,Premier répondant,-73.574247,45.494499,97.666667
3,2015-01-01 00:08:34,Appel de Cie de détection,Alarmes-incendies,-73.665779,45.442005,60.000000
4,2015-01-01 00:11:28,Premier répondant,Premier répondant,-73.489981,45.641294,94.500000
5,2015-01-01 00:14:06,Odeur suspecte - gaz,Sans incendie,-73.615028,45.483883,44.500000
6,2015-01-01 00:20:27,Premier répondant,Premier répondant,-73.580606,45.485622,29.000000
7,2015-01-01 00:21:01,Premier répondant,Premier répondant,-73.594318,45.584117,42.000000
8,2015-01-01 00:22:38,Alarme vérification,Alarmes-incendies,-73.641177,45.597359,42.000000
9,2015-01-01 00:24:18,10-22 sans feu,Sans incendie,-73.637632,45.420985,49.600000


In [0]:
alert_house_30_mean.describe()

,incident_long,incident_lat,mean_age
count,339365.000000,339365.000000,339365.000000
mean,-73.618531,45.527448,60.080318
std,0.079659,0.055932,27.219830
min,-73.967681,45.402687,0.000000
25%,-73.640738,45.489181,43.000000
50%,-73.601556,45.524454,57.666667
75%,-73.568947,45.562622,77.000000
max,-73.479402,45.701779,217.000000


In [0]:
alert_house_30_mean.to_csv(path + 'incident_mean_5_in_50meters.csv')